In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
! pip install impyute

In [5]:
from impyute.imputation.cs import mice
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [6]:
import lightgbm as lgb
from sklearn.metrics import *
from sklearn.model_selection import KFold

In [7]:
train_err = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_err_data.csv')
train_problem = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_problem_data.csv')
test_err = pd.read_csv('/content/drive/MyDrive/데이콘/data/test_err_data.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/데이콘/data/sample_submission.csv')
train_quality = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_quality_data.csv')
test_quality = pd.read_csv('/content/drive/MyDrive/데이콘/data/test_quality_data.csv')

In [ ]:
train_err.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [ ]:
train_quality.head()

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0


In [ ]:
train_quality['user_id'].value_counts()

15357    4224
23065    3828
22632    3744
11635    3504
16147    3336
         ... 
11014      12
24847      12
12282      12
18186      12
18698      12
Name: user_id, Length: 8281, dtype: int64

In [ ]:
train_quality[train_quality['user_id']==18698]

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
470784,20201106223000,18698,04.16.3553,0.0,0,0.0,0,0,0,0,0,0,0,2,0,0
470785,20201106223000,18698,04.16.3553,0.0,0,0.0,0,0,0,0,0,0,0,2,0,0
470786,20201106223000,18698,04.16.3553,0.0,0,0.0,0,0,0,0,0,0,0,2,0,0
470787,20201106223000,18698,04.16.3553,0.0,0,0.0,0,0,0,0,0,0,0,2,0,0
470788,20201106223000,18698,04.16.3553,0.0,0,0.0,0,0,0,0,0,0,0,2,0,0
470789,20201106223000,18698,04.16.3553,0.0,0,0.0,0,0,0,0,0,0,0,2,0,0
470790,20201106223000,18698,04.16.3553,0.0,0,0.0,0,0,0,0,0,0,0,2,0,0
470791,20201106223000,18698,04.16.3553,0.0,0,0.0,0,0,2,0,0,0,0,2,0,0
470792,20201106223000,18698,04.16.3553,0.0,0,0.0,0,0,0,0,0,0,0,2,0,0
470793,20201106223000,18698,04.16.3553,0.0,0,0.0,0,0,0,0,0,0,0,2,0,0


In [ ]:
train_quality[train_quality['user_id']==11014]

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
52884,20201114014000,11014,04.33.1261,0.0,0,0.0,0,0,1,0,0,0,0,4,0,0
52885,20201114014000,11014,04.33.1261,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
52886,20201114014000,11014,04.33.1261,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
52887,20201114014000,11014,04.33.1261,0.0,0,0.0,0,0,1,0,0,0,0,4,0,0
52888,20201114014000,11014,04.33.1261,-1.0,-1,-1.0,0,0,-1,-1,0,0,0,4,-1,0
52889,20201114014000,11014,04.33.1261,-1.0,-1,-1.0,0,0,-1,-1,0,0,0,4,-1,0
52890,20201114014000,11014,04.33.1261,-1.0,-1,-1.0,0,0,-1,-1,0,0,0,4,-1,0
52891,20201114014000,11014,04.33.1261,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
52892,20201114014000,11014,04.33.1261,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
52893,20201114014000,11014,04.33.1261,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0


* train_quality에서 user_id = 11014인 행만 추출한 결과 quality_0,1,2,5,6,11은 값이 같았고 quality10에서는 모두 4로 값이 같음을 알 수 있다. 

In [ ]:
train_quality[train_quality['user_id']==12282]

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
120552,20201117232000,12282,04.22.1750,-1.0,-1,-1.0,0,0,-1,-1,0,0,0,4,-1,0
120553,20201117232000,12282,04.22.1750,-1.0,-1,-1.0,0,0,-1,-1,0,0,0,4,-1,0
120554,20201117232000,12282,04.22.1750,-1.0,-1,-1.0,0,0,-1,-1,0,0,0,4,-1,0
120555,20201117232000,12282,04.22.1750,-1.0,-1,-1.0,0,0,-1,-1,0,0,0,4,-1,0
120556,20201117232000,12282,04.22.1750,-1.0,-1,-1.0,0,0,-1,-1,0,0,0,4,-1,0
120557,20201117232000,12282,04.22.1750,-1.0,-1,-1.0,0,0,-1,-1,0,0,0,4,-1,0
120558,20201117232000,12282,04.22.1750,-1.0,-1,-1.0,0,0,-1,-1,0,0,0,4,-1,0
120559,20201117232000,12282,04.22.1750,-1.0,-1,-1.0,0,0,-1,-1,0,0,0,4,-1,0
120560,20201117232000,12282,04.22.1750,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
120561,20201117232000,12282,04.22.1750,0.0,0,0.0,0,0,3,0,0,0,0,4,0,0


In [ ]:
train_quality[train_quality['user_id']==20000]

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
535476,20201121021000,20000,05.15.2138,2.0,0,1.0,0,0,0,0,0,0,2,1,0,0
535477,20201121021000,20000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,2,1,0,0
535478,20201121021000,20000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,2,1,0,0
535479,20201121021000,20000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,2,1,0,0
535480,20201121021000,20000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,2,1,0,0
535481,20201121021000,20000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,2,1,0,0
535482,20201121021000,20000,05.15.2138,2.0,0,1.0,0,0,1,0,0,0,2,1,0,0
535483,20201121021000,20000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,2,1,0,0
535484,20201121021000,20000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,2,1,0,0
535485,20201121021000,20000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,2,1,0,0


In [ ]:
train_quality['time'].dtype

dtype('int64')

In [ ]:
train_err_id_18696 = train_err[train_err['user_id']==18698]
train_err_id_18696[train_err_id_18696['time'] == 20201106223000] 

,user_id,time,model_nm,fwver,errtype,errcode


* train_err와 train_quality 데이터의 id는 같아도 time이 겹치지는 않는다. 

In [ ]:
train_quality_id_15357 = train_quality[train_quality['user_id']==15357]
train_quality_id_15357['time'].value_counts() # 352개의 다른 time이 존재, 한 time당 12개씩 존재

train_err_id_15357 = train_err[train_err['user_id']==15357]
train_err_id_15357['time'].value_counts() # 한 time당 규칙적으로 개수가 일정하지 않음

id_15357_quality_list = train_quality_id_15357['time'].unique()
id_15357_err_list = train_err_id_15357['time'].unique()

intersection = list(set(id_15357_quality_list)&set(id_15357_err_list))
intersection # id=15357인 quality 데이터와 err데이터의 겹치는 time이 존재하지 않음

[]

* train_err_data와 train_quality_data의 user_id에 따른 time이 겹치지않는 데이터들이 존재합니다. 

In [ ]:
train_quality_id_16147 = train_quality[train_quality['user_id']==16147]
train_quality_id_16147['time'].value_counts() # 278개의 다른 time이 존재, 한 time당 12개씩 존재

train_err_id_16147 = train_err[train_err['user_id']==16147]
train_err_id_16147['time'].value_counts() # 한 time당 규칙적으로 개수가 일정하지 않음

id_16147_quality_list = train_quality_id_16147['time'].unique()
id_16147_err_list = train_err_id_16147['time'].unique()

intersection = list(set(id_16147_quality_list)&set(id_16147_err_list))
intersection # id=15357인 quality 데이터와 err데이터의 겹치는 time이 1개 존재함

[20201106081000]

* train_err_data와 train_quality_data의 user_id에 따른 time이 겹치는 데이터가 존재하기도 한다. 

In [ ]:
# train_quality의 time을 날짜 시간 초별로 구분하기
train_quality.rename(columns={"time": "original_time"}, inplace=True)
train_quality['DT'] = pd.to_datetime(train_quality['original_time'], format='%Y%m%d%H%M%S')
train_quality['date'] = train_quality['DT'].apply(lambda x: x.strftime('%Y-%m-%d'))
train_quality['time'] = train_quality['DT'].apply(lambda x: x.strftime('%H:%M:%S'))
train_quality['hour'] = train_quality['DT'].apply(lambda x: x.strftime('%H'))
train_quality.head()

,original_time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,DT,date,time,hour
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00,2020-11-29,09:00:00,09
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00,2020-11-29,09:00:00,09
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00,2020-11-29,09:00:00,09
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00,2020-11-29,09:00:00,09
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00,2020-11-29,09:00:00,09


In [ ]:
train_err.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [ ]:
train_err.rename(columns={"time": "original_time"}, inplace=True)
train_err['DT'] = pd.to_datetime(train_err['original_time'], format='%Y%m%d%H%M%S')
train_err['date'] = train_err['DT'].apply(lambda x: x.strftime('%Y-%m-%d'))
train_err['time'] = train_err['DT'].apply(lambda x: x.strftime('%H:%M:%S'))
train_err['hour'] = train_err['DT'].apply(lambda x: x.strftime('%H'))
train_err.head()

,user_id,original_time,model_nm,fwver,errtype,errcode,DT,date,time,hour
0,10000,20201101025616,model_3,05.15.2138,15,1,2020-11-01 02:56:16,2020-11-01,02:56:16,02
1,10000,20201101030309,model_3,05.15.2138,12,1,2020-11-01 03:03:09,2020-11-01,03:03:09,03
2,10000,20201101030309,model_3,05.15.2138,11,1,2020-11-01 03:03:09,2020-11-01,03:03:09,03
3,10000,20201101050514,model_3,05.15.2138,16,1,2020-11-01 05:05:14,2020-11-01,05:05:14,05
4,10000,20201101050515,model_3,05.15.2138,4,0,2020-11-01 05:05:15,2020-11-01,05:05:15,05


In [ ]:
train_err[train_err['user_id'] == 10000]['original_time'].value_counts()

20201119223331    4
20201107110543    4
20201118163318    4
20201103140502    4
20201107070542    3
                 ..
20201110021658    1
20201122090120    1
20201120075804    1
20201113051136    1
20201115000313    1
Name: original_time, Length: 285, dtype: int64

In [ ]:
train_err[(train_err['original_time'] == 20201119223331) & (train_err['user_id'] == 10000)]

,user_id,original_time,model_nm,fwver,errtype,errcode,DT,date,time,hour
213,10000,20201119223331,model_3,05.15.2138,14,1,2020-11-19 22:33:31,2020-11-19,22:33:31,22
214,10000,20201119223331,model_3,05.15.2138,10,1,2020-11-19 22:33:31,2020-11-19,22:33:31,22
215,10000,20201119223331,model_3,05.15.2138,12,1,2020-11-19 22:33:31,2020-11-19,22:33:31,22
216,10000,20201119223331,model_3,05.15.2138,11,1,2020-11-19 22:33:31,2020-11-19,22:33:31,22


* train_err 데이터에서 user_id가 10000인 original_time이 같은 경우의 데이터프레임을 추출했다. 
    * 날짜시간이 모두 같은 데이터인데 errtype만 14,10,12,11으로 다르게 나왔다.
    * 같은 기기에서 동시간에 다른 errtype이 나올 수 있는 이유는 무엇일까?   


In [ ]:
train_err['model_nm'].value_counts()

model_1    5384491
model_0    4176279
model_2    3473254
model_3    1907438
model_4    1507802
model_8      36064
model_7      35356
model_5      32233
model_6       1746
Name: model_nm, dtype: int64

- 에러가 가장 많이 발생한 모델은 model_1이며 가장 적게 발생한 모델은 model_6이다. 

In [ ]:
train_err['errtype'].value_counts()

23    2276515
31    1985294
22    1865361
15    1604951
4     1296750
16    1249376
32    1130564
5      952920
40     730378
28     544306
27     540804
26     536401
12     320181
11     307030
14     252015
33     178477
34     165536
10     133403
41     104138
7       51993
6       50482
17      46009
24      39071
42      34374
3       30758
2       26106
13      22843
1       21545
39      15820
35      10187
37       9336
36       9317
38       4539
18       2671
20       2080
25       1184
19        664
21        607
30        402
8         166
9         109
Name: errtype, dtype: int64

- 에러가 가장 많이 발생한 errtypq은 23이고 가장 적게 발생한 errtype은 9이다.   

In [ ]:
train_err['errcode'].value_counts()

1                     8906967
0                     2599123
connection timeout    1835600
B-A8002                897863
80                     334018
                       ...   
14154                       1
6005                        1
10827                       1
3429                        1
8414                        1
Name: errcode, Length: 2805, dtype: int64

- errcode 값은 매우 다양한 형태로 나타났다. 
    * 차례로 숫자 1, 0, connection timeout이 많이 발생한 것을 알 수 있다.  

In [ ]:
train_err[train_err['errtype']==23]['date'].value_counts()

2020-11-07    100059
2020-11-01     97298
2020-11-08     96143
2020-11-15     90408
2020-11-06     82230
2020-11-28     82159
2020-11-02     81898
2020-11-09     81875
2020-11-14     79388
2020-11-05     79077
2020-11-03     78948
2020-11-10     78200
2020-11-04     77836
2020-11-22     74894
2020-11-29     73336
2020-11-12     72884
2020-11-11     72762
2020-11-16     72481
2020-11-13     70951
2020-11-21     70811
2020-11-23     70761
2020-11-17     69494
2020-11-19     69141
2020-11-18     68014
2020-11-27     67774
2020-11-20     67593
2020-11-26     67402
2020-11-25     63230
2020-11-30     60358
2020-11-24     59108
2020-12-01         2
Name: date, dtype: int64

In [ ]:
train_err[train_err['errtype']==23]['hour'].value_counts()

20    149707
21    147524
19    142923
22    137470
18    123348
23    118562
17    106342
16     96127
13     93742
12     92713
15     92011
00     90924
14     90505
09     89938
10     89280
11     88646
08     86004
01     72663
02     70801
03     67488
07     65812
04     61076
05     51706
06     51203
Name: hour, dtype: int64

In [ ]:
train_err[train_err['errtype']==31]['date'].value_counts()

2020-11-22    69939
2020-11-18    69795
2020-11-29    69270
2020-11-25    69200
2020-11-15    69042
2020-11-19    68840
2020-11-30    68093
2020-11-07    67370
2020-11-28    67065
2020-11-20    67040
2020-11-24    66873
2020-11-08    66860
2020-11-23    66638
2020-11-27    66614
2020-11-21    66098
2020-11-12    65925
2020-11-13    65687
2020-11-11    65340
2020-11-14    65170
2020-11-26    65153
2020-11-16    65126
2020-11-17    65088
2020-11-09    64869
2020-11-05    64689
2020-11-10    64636
2020-11-06    64021
2020-11-01    63810
2020-11-02    63010
2020-11-04    62287
2020-11-03    61745
2020-12-01        1
Name: date, dtype: int64

In [ ]:
train_err[train_err['errtype']==31]['hour'].value_counts()

18    130920
17    123603
19    122564
08    111260
20    107548
16    104983
07    104694
21    104151
09     96318
22     89679
15     89597
12     85926
13     84488
14     83419
10     82871
11     80087
23     65447
06     62428
00     47978
05     45654
04     43906
02     42861
03     41694
01     33218
Name: hour, dtype: int64

In [ ]:
train_err['hour'].value_counts()

19    899350
18    848590
20    848529
21    846325
22    774901
17    767757
00    722370
08    718565
16    701977
02    691168
03    686960
09    685861
23    684248
04    680424
13    660555
15    653461
07    653261
12    646229
10    643472
14    635799
11    629336
05    554346
06    470877
01    450302
Name: hour, dtype: int64

- 에러가 가장 많이 발생하는 시간대는 저녁(18~22)라는 것을 알 수 있고 가장 적게 발생하는 시간대는 새벽이나 오전인 것으로 나타났다. 

In [ ]:
train_err['date'].value_counts()

2020-11-21    649909
2020-11-22    596759
2020-11-28    594271
2020-11-15    591868
2020-11-24    590701
2020-11-01    589297
2020-11-29    588534
2020-11-07    587836
2020-11-08    581062
2020-11-14    574420
2020-11-06    558375
2020-11-19    557735
2020-11-23    550252
2020-11-18    539401
2020-11-09    539332
2020-11-02    539259
2020-11-20    533996
2020-11-25    531097
2020-11-05    529307
2020-11-27    528339
2020-11-03    528263
2020-11-26    526613
2020-11-12    526538
2020-11-10    523930
2020-11-16    522448
2020-11-04    522174
2020-11-11    519095
2020-11-17    516087
2020-11-30    512635
2020-11-13    505044
2020-12-01        78
2020-10-31         5
2020-12-02         3
Name: date, dtype: int64

In [ ]:
train_quality.head(2)

,original_time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,DT,date,time,hour
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00,2020-11-29,09:00:00,09
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00,2020-11-29,09:00:00,09


In [ ]:
train_quality['hour'].value_counts()

22    59352
20    54132
21    52440
23    46860
18    43104
19    42348
00    41808
14    41292
16    36408
12    36012
15    36012
10    35592
17    35052
13    32400
01    31080
11    30744
09    28260
02    27036
08    25704
03    22968
04    21024
07    17472
05    16440
06    15084
Name: hour, dtype: int64

- train_err 데이터셋과 비슷하게 quality 측정 시간은 저녁 (21~23시)때가 많고 새벽이 가장 적음을 알 수 있다. 

In [ ]:
# train_quality, train_err의 user_id가 겹치는가?
quality_user_id = train_quality['user_id'].unique()
err_user_id = train_err['user_id'].unique()
intersection = list(set(quality_user_id) & set(err_user_id))
len(intersection)

8281

- 에러가 발생한 user_id와 퀄리티를 측정한 user_id가 약 15000개 중 8281개가 겹친다. 

In [ ]:
len(quality_user_id)

8281

In [ ]:
train_err = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_err_data.csv')
train_problem = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_problem_data.csv')
test_err = pd.read_csv('/content/drive/MyDrive/데이콘/data/test_err_data.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/데이콘/data/sample_submission.csv')
train_quality = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_quality_data.csv')
test_quality = pd.read_csv('/content/drive/MyDrive/데이콘/data/test_quality_data.csv')

In [ ]:
train_err.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [ ]:
train_quality.head()

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0


In [8]:
train_quality_df = train_quality.drop(['time','fwver'],axis=1)
train_quality_df.head()

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,10000,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,10000,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,10000,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,10000,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0


In [9]:
# train_quality 데이터 user_id별 time,fwver와 상관없이 quality변수값 평균화
mean_train_quality = train_quality_df.groupby('user_id').agg('mean')
mean_train_quality = mean_train_quality.reset_index()
mean_train_quality

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_6,quality_11,quality_12
0,10000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,10002,0.000000,-0.020833,-0.010417,0.0,0.0,0.458333,-0.020833,0.0
2,10004,-0.083333,-0.083333,-0.083333,0.0,0.0,3.541667,-0.083333,0.0
3,10005,-0.416667,-0.416667,-0.416667,0.0,0.0,1.083333,-0.416667,0.0
4,10006,0.000000,0.000000,0.000000,0.0,0.0,0.111111,0.000000,0.0
...,...,...,...,...,...,...,...,...,...
8276,24990,NaN,-0.333333,-0.333333,0.0,0.0,-0.333333,-0.333333,0.0
8277,24992,-0.333333,-0.333333,-0.333333,0.0,0.0,-0.333333,-0.333333,0.0
8278,24993,-0.291667,-0.291667,-0.291667,0.0,0.0,-0.291667,-0.291667,0.0
8279,24995,-0.416667,-0.416667,-0.416667,0.0,0.0,-0.416667,-0.416667,0.0


* train_quality와 train_problem의 user_id간의 상관관계가 있을까? 
  - 불만을 표현한 유저들이 quality가 생성된 기기를 사용하는 유저들과 관계가 있는지 알아보자.

In [ ]:
train_quality_user_id = train_quality['user_id'].unique()
train_problem_user_id = train_problem['user_id'].unique()
intersection = list(set(train_quality_user_id)&set(train_problem_user_id))
len(intersection)

3167

In [ ]:
len(train_problem_user_id)

5000

In [ ]:
print(round(3176/5000*100),'%')

64 %


* 5000명의 불만을 제기한 유저 중 3167명 즉 약 64%가 quality 데이터셋에 존재한 유저이다.
  - 불만을 제기한 유저 중 절반이상이 quality 유저와 같다는 건 무엇을 의미할까?

In [ ]:
train_quality_df.describe()

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_6,quality_11,quality_12
count,828624.000000,684192.000000,828624.000000,788511.000000,828624.0,828624.0,828624.000000,828624.000000,828624.000000
mean,17574.678315,4.148701,-0.171782,4.751094,0.0,0.0,2.043391,-0.181638,0.045878
std,4374.113554,479.315029,0.692386,586.252469,0.0,0.0,32.695380,0.397767,0.302452
min,10000.000000,-1.000000,-1.000000,-1.000000,0.0,0.0,-1.000000,-1.000000,0.000000
25%,13685.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
50%,17597.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
75%,21423.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
max,24997.000000,157667.000000,171.000000,191859.000000,0.0,0.0,600.000000,14.000000,14.000000


In [ ]:
mean_train_quality.describe()

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_6,quality_11,quality_12
count,8281.000000,7727.000000,8281.000000,8224.000000,8281.0,8281.0,8281.000000,8281.000000,8281.000000
mean,17515.508996,3.192233,-0.188322,3.270154,0.0,0.0,2.178745,-0.200108,0.017214
std,4322.740630,105.559719,0.219599,103.006859,0.0,0.0,18.066568,0.210423,0.140654
min,10000.000000,-0.916667,-0.916667,-0.916667,0.0,0.0,-0.916667,-0.916667,0.000000
25%,13763.000000,-0.333333,-0.309524,-0.333333,0.0,0.0,-0.256410,-0.333333,0.000000
50%,17539.000000,-0.151235,-0.138889,-0.145833,0.0,0.0,-0.083333,-0.152542,0.000000
75%,21282.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
max,24997.000000,5409.750000,3.797619,5409.750000,0.0,0.0,464.666667,0.333333,4.571429


* train_quality 데이터의 quality_3, quality_4의 값은 모두 0으로 삭제해도 무방할 것으로 보인다.

In [ ]:
train_quality_df.isnull().sum()

user_id            0
quality_0     144432
quality_1          0
quality_2      40113
quality_3          0
quality_4          0
quality_5         20
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

* train_quality 데이터의 quality_0, quality_2, quality_5에 결측치가 약 14만,4만,20개가 존재한다.

In [ ]:
len(train_quality)

828624

In [ ]:
print(len(train_quality[train_quality['quality_0']==0]))
print(round(542790/828624*100),'%')
print(len(train_quality[train_quality['quality_0']==-1]))
print(round(130828/828624*100),'%')

542790
66 %
130828
16 %


* quality_0이 0인 갯수는 약 54만건이므로 데이터의 66%를 차지하고 -1인 갯수는 약 13만건으로 데이터의 16%를 차지한다. 
  * 즉, quality_0값이 0과 -1인 갯수의 합은 전체 데이터의 82%를 차지한다.


In [ ]:
print(len(train_quality[train_quality['quality_2']==0]))
print(round(632469/828624*100),'%')
print(len(train_quality[train_quality['quality_2']==-1]))
print(round(144392/828624*100),'%')

632469
76 %
144392
17 %


* quality_0과 마찬가지로 quality_2가 0과 -1인 갯수의 합은 전체 데이터의 무러 93%를 차지한다.

In [ ]:
print(len(train_quality[train_quality['quality_5']==0]))
print(round(428096/828624*100),'%')
print(len(train_quality[train_quality['quality_5']==-1]))
print(round(153354/828624*100),'%')

0
52 %
0
19 %


* quality_0과 quality_2와 마찬가지로 quality_5도 0과 -1인 갯수의 합은 전체 데이터의 무러 61%%를 차지한다. 

In [ ]:
# train_quality 결측치 최빈값 0으로 대체 
# train_quality_df.fillna(0,inplace=True)
# train_quality_df.isnull().sum()

In [ ]:
train_quality_df.dtypes

user_id         int64
quality_0     float64
quality_1       int64
quality_2     float64
quality_3       int64
quality_4       int64
quality_5      object
quality_6       int64
quality_7      object
quality_8      object
quality_9      object
quality_10     object
quality_11      int64
quality_12      int64
dtype: object

In [11]:
def string2num(x):
  x = re.sub(r"[^0-9]+",'',str(x))
  if x == '':
    return 0
  else:
    return int(x)

In [12]:
train_quality_df['quality_5'] = train_quality_df['quality_5'].apply(lambda x : string2num(x))
train_quality_df['quality_7'] = train_quality_df['quality_7'].apply(lambda x : string2num(x))
train_quality_df['quality_8'] = train_quality_df['quality_8'].apply(lambda x : string2num(x))
train_quality_df['quality_9'] = train_quality_df['quality_9'].apply(lambda x : string2num(x))
train_quality_df['quality_10'] = train_quality_df['quality_10'].apply(lambda x : string2num(x))

In [13]:
train_quality_df['quality_5'] = train_quality_df['quality_5'].astype('int')
train_quality_df['quality_7'] = train_quality_df['quality_7'].astype('int')
train_quality_df['quality_8'] = train_quality_df['quality_8'].astype('int')
train_quality_df['quality_9'] = train_quality_df['quality_9'].astype('int')
train_quality_df['quality_10'] = train_quality_df['quality_10'].astype('int')

In [14]:
mean_train_quality = train_quality_df.groupby('user_id').agg('mean')
mean_train_quality = mean_train_quality.reset_index()
mean_train_quality

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,0.000000,0.000000,0.000000,0.0,0.0,0.500000,0.000000,0.000000,0.0,0.000,6.000000,0.000000,0.0
1,10002,0.000000,-0.020833,-0.010417,0.0,0.0,0.343750,0.458333,5.750000,0.0,0.125,3.875000,-0.020833,0.0
2,10004,-0.083333,-0.083333,-0.083333,0.0,0.0,0.250000,3.541667,43.500000,0.0,0.000,2.000000,-0.083333,0.0
3,10005,-0.416667,-0.416667,-0.416667,0.0,0.0,0.833333,1.083333,18.000000,0.0,0.000,5.000000,-0.416667,0.0
4,10006,0.000000,0.000000,0.000000,0.0,0.0,0.333333,0.111111,1.333333,0.0,0.000,4.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8276,24990,NaN,-0.333333,-0.333333,0.0,0.0,0.416667,-0.333333,0.000000,0.0,0.000,1.000000,-0.333333,0.0
8277,24992,-0.333333,-0.333333,-0.333333,0.0,0.0,0.861111,-0.333333,0.000000,0.0,0.000,6.333333,-0.333333,0.0
8278,24993,-0.291667,-0.291667,-0.291667,0.0,0.0,0.625000,-0.291667,0.000000,0.0,0.000,4.000000,-0.291667,0.0
8279,24995,-0.416667,-0.416667,-0.416667,0.0,0.0,0.625000,-0.416667,0.000000,0.0,0.000,2.500000,-0.416667,0.0


In [15]:
mean_train_quality = pd.DataFrame(IterativeImputer(verbose=False).fit_transform(mean_train_quality))
mean_train_quality

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,10000.0,0.000000,0.000000,0.000000,0.0,0.0,0.500000,0.000000,0.000000,0.0,0.000,6.000000,0.000000,0.0
1,10002.0,0.000000,-0.020833,-0.010417,0.0,0.0,0.343750,0.458333,5.750000,0.0,0.125,3.875000,-0.020833,0.0
2,10004.0,-0.083333,-0.083333,-0.083333,0.0,0.0,0.250000,3.541667,43.500000,0.0,0.000,2.000000,-0.083333,0.0
3,10005.0,-0.416667,-0.416667,-0.416667,0.0,0.0,0.833333,1.083333,18.000000,0.0,0.000,5.000000,-0.416667,0.0
4,10006.0,0.000000,0.000000,0.000000,0.0,0.0,0.333333,0.111111,1.333333,0.0,0.000,4.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8276,24990.0,-0.290489,-0.333333,-0.333333,0.0,0.0,0.416667,-0.333333,0.000000,0.0,0.000,1.000000,-0.333333,0.0
8277,24992.0,-0.333333,-0.333333,-0.333333,0.0,0.0,0.861111,-0.333333,0.000000,0.0,0.000,6.333333,-0.333333,0.0
8278,24993.0,-0.291667,-0.291667,-0.291667,0.0,0.0,0.625000,-0.291667,0.000000,0.0,0.000,4.000000,-0.291667,0.0
8279,24995.0,-0.416667,-0.416667,-0.416667,0.0,0.0,0.625000,-0.416667,0.000000,0.0,0.000,2.500000,-0.416667,0.0


In [28]:
mean_train_quality = mean_train_quality.rename(columns={0:'user_id',
                                   1:'quality_0',
                                   2:'quality_1',
                                   3:'quality_2',
                                   4:'quality_3',
                                   5:'quality_4',
                                   6:'quality_5',
                                   7:'quality_6',
                                   8:'quality_7',
                                   9:'quality_8',
                                   10:'quality_9',
                                   11:'quality_10',
                                   12:'quality_11',
                                   13:'quality_12'})

In [16]:
mean_train_quality.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64

In [22]:
mean_train_quality.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,8281.000000,8281.000000,8281.000000,8281.000000,8281.0,8281.0,8281.000000,8281.000000,8281.000000,8281.000000,8281.000000,8281.000000,8281.000000,8281.000000
mean,17515.508996,3.160927,-0.188322,3.246128,0.0,0.0,63.950054,2.178745,28.563257,0.158620,41.357528,764.982573,-0.200108,0.017214
std,4322.740630,102.446494,0.219599,102.652101,0.0,0.0,1035.677913,18.066568,216.655614,0.901582,1231.877870,12428.098464,0.210423,0.140654
min,10000.000000,-0.916667,-0.916667,-0.916667,0.0,0.0,0.000000,-0.916667,0.000000,0.000000,0.000000,0.000000,-0.916667,0.000000
25%,13763.000000,-0.305556,-0.309524,-0.333333,0.0,0.0,0.333333,-0.256410,0.000000,0.000000,0.000000,2.500000,-0.333333,0.000000
50%,17539.000000,-0.166667,-0.138889,-0.147321,0.0,0.0,0.611111,-0.083333,0.000000,0.000000,0.000000,4.000000,-0.152542,0.000000
75%,21282.000000,0.000000,0.000000,0.000000,0.0,0.0,1.277778,0.000000,0.214286,0.000000,0.000000,13.000000,0.000000,0.000000
max,24997.000000,5409.750000,3.797619,5409.750000,0.0,0.0,57637.750000,464.666667,5576.000000,47.071429,64928.000000,691646.000000,0.333333,4.571429


In [17]:
id_error = train_err[['user_id','errtype']].values
error = np.zeros((15000,42))

for person_idx, err in tqdm(id_error):
  error[person_idx-10000,err-1]+=1
error.shape

100%|██████████| 16554663/16554663 [00:46<00:00, 356503.68it/s]


(15000, 42)

In [18]:
df_error = pd.DataFrame(error)
df_error['user_id'] = train_err['user_id']
df_error

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,user_id
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10000
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,10000
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10000
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,10000
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0,10011
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10011
14997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,0.0,0.0,0.0,50.0,29.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0,223.0,0.0,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0,10011
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,14.0,0.0,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0,10011


In [ ]:
len(df_error)

15000

In [29]:
merged_train_err_quality = pd.merge(df_error,mean_train_quality,on='user_id',how='left')
del merged_train_err_quality['user_id']
merged_train_err_quality.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,6.0,0.0,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,6.0,0.0,0.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,6.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,6.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,6.0,0.0,0.0


In [30]:
len(merged_train_err_quality)

15000

In [31]:
multiple_imputation_df = pd.DataFrame(IterativeImputer(verbose=False).fit_transform(merged_train_err_quality))
multiple_imputation_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,0.000000,0.000000,0.00000,0.0,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,0.000000,0.000000,0.00000,0.0,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0,-0.592126,-0.531088,-0.59244,0.0,0.0,0.801037,-0.184755,4.895993,0.740001,0.003769,2.499384,-0.592755,0.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.592126,-0.531088,-0.59244,0.0,0.0,0.801037,-0.184755,4.895993,0.739994,0.003768,2.499383,-0.592754,0.0
14997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,0.0,0.0,0.0,50.0,29.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0,223.0,0.0,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0,-0.592126,-0.531088,-0.59244,0.0,0.0,0.801037,-0.184755,4.895993,0.739998,0.003768,2.499384,-0.592754,0.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,14.0,0.0,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0,-0.592126,-0.531088,-0.59244,0.0,0.0,0.801037,-0.184755,4.895993,0.739998,0.003768,2.499384,-0.592754,0.0


In [32]:
problem = np.zeros(15000)
problem[train_problem.user_id.unique()-10000] = 1 
problem.shape

(15000,)

In [33]:
train_x = multiple_imputation_df.values
train_y = problem
print(train_x.shape)
print(train_y.shape)

(15000, 55)
(15000,)


In [34]:
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 3
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

Training until validation scores don't improve for 3 rounds.
[20]	valid_0's auc: 0.797138	valid_0's pr_auc: 0.794406
Early stopping, best iteration is:
[22]	valid_0's auc: 0.79802	valid_0's pr_auc: 0.795595
Training until validation scores don't improve for 3 rounds.
Early stopping, best iteration is:
[2]	valid_0's auc: 0.78607	valid_0's pr_auc: 0.318309
Training until validation scores don't improve for 3 rounds.
Early stopping, best iteration is:
[2]	valid_0's auc: 0.758098	valid_0's pr_auc: 0.35982
Training until validation scores don't improve for 3 rounds.
Early stopping, best iteration is:
[11]	valid_0's auc: 0.805233	valid_0's pr_auc: 0.548772
Training until validation scores don't improve for 3 rounds.
Early stopping, best iteration is:
[9]	valid_0's auc: 0.800653	valid_0's pr_auc: 0.974202


In [35]:
print(np.mean(auc_scores))

0.789614722838928


In [36]:
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [37]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

100%|██████████| 16532648/16532648 [00:47<00:00, 348480.73it/s]

(14999, 42)


In [38]:
test_quality_df = test_quality.drop(['time','fwver'],axis=1)

In [ ]:
test_quality_df.isnull().sum()

In [ ]:
len(test_quality_df)

747972

In [ ]:
print(len(test_quality[test_quality['quality_0']==0]))
print(round(505710/747972*100),'%')
print(len(test_quality[test_quality['quality_0']==-1]))
print(round(127001/747972*100),'%')

505710
61 %
127001
15 %


In [ ]:
print(len(test_quality[test_quality['quality_1']==0]))
print(round(576597/747972*100),'%')
print(len(test_quality[test_quality['quality_1']==-1]))
print(round(134061/747972*100),'%')

576597
77 %
134061
18 %


In [ ]:
print(len(test_quality[test_quality['quality_2']==0]))
print(round(578838/747972*100),'%')
print(len(test_quality[test_quality['quality_2']==-1]))
print(round(136864/747972*100),'%')

578838
77 %
136864
18 %


In [ ]:
print(len(test_quality[test_quality['quality_5']==0]))
print(round(399032/747972*100),'%')
print(len(test_quality[test_quality['quality_5']==-1]))
print(round(141788/747972*100),'%')

0
53 %
0
19 %


In [ ]:
test_quality_df.dtypes

user_id         int64
quality_0     float64
quality_1      object
quality_2     float64
quality_3       int64
quality_4       int64
quality_5      object
quality_6       int64
quality_7      object
quality_8      object
quality_9      object
quality_10     object
quality_11      int64
quality_12      int64
dtype: object

In [39]:
test_quality_df['quality_1'] = test_quality_df['quality_1'].apply(lambda x : string2num(x))
test_quality_df['quality_5'] = test_quality_df['quality_5'].apply(lambda x : string2num(x))
test_quality_df['quality_7'] = test_quality_df['quality_7'].apply(lambda x : string2num(x))
test_quality_df['quality_8'] = test_quality_df['quality_8'].apply(lambda x : string2num(x))
test_quality_df['quality_9'] = test_quality_df['quality_9'].apply(lambda x : string2num(x))
test_quality_df['quality_10'] = test_quality_df['quality_10'].apply(lambda x : string2num(x))

In [40]:
test_quality_df['quality_1'] = test_quality_df['quality_1'].astype('int')
test_quality_df['quality_5'] = test_quality_df['quality_5'].astype('int')
test_quality_df['quality_7'] = test_quality_df['quality_7'].astype('int')
test_quality_df['quality_8'] = test_quality_df['quality_8'].astype('int')
test_quality_df['quality_9'] = test_quality_df['quality_9'].astype('int')
test_quality_df['quality_10'] = test_quality_df['quality_10'].astype('int')

In [41]:
test_quality_df.head()

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
1,30000,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
2,30000,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
3,30000,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
4,30000,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0


In [42]:
mean_test_quality = test_quality_df.groupby('user_id').agg('mean')
mean_test_quality = mean_test_quality.reset_index()
mean_test_quality

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,0.000000,0.000000,0.000000,0.0,0.0,0.166667,0.416667,5.00,0.0,0.00,2.000000,0.000000,0.0
1,30001,-0.250000,0.250000,-0.250000,0.0,0.0,71.444444,-0.250000,0.00,0.0,0.00,854.333333,-0.250000,0.0
2,30002,-0.125000,0.125000,-0.125000,0.0,0.0,1.233333,-0.125000,0.00,0.0,0.00,13.300000,-0.125000,0.0
3,30003,0.000000,0.000000,0.000000,0.0,0.0,0.200000,3.666667,44.00,0.0,0.00,2.400000,0.000000,0.0
4,30004,-0.250000,0.250000,-0.250000,0.0,0.0,0.333333,1.125000,16.50,0.0,0.00,1.000000,-0.250000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8263,44990,-0.111111,0.111111,-0.111111,0.0,0.0,1.500000,-0.111111,0.00,0.0,0.00,16.666667,-0.111111,0.0
8264,44993,0.145833,0.041667,0.145833,0.0,0.0,0.333333,2.229167,27.25,0.0,2.25,3.500000,-0.041667,0.0
8265,44994,-0.916667,0.916667,-0.916667,0.0,0.0,1.083333,-0.916667,0.00,0.0,0.00,2.000000,-0.916667,0.0
8266,44996,-0.046296,0.046296,-0.046296,0.0,0.0,0.388889,-0.046296,0.00,0.0,0.00,4.111111,-0.046296,0.0


In [44]:
mean_test_quality.isnull().sum()

user_id         0
quality_0     545
quality_1       0
quality_2      43
quality_3       0
quality_4       0
quality_5       0
quality_6       0
quality_7       0
quality_8       0
quality_9       0
quality_10      0
quality_11      0
quality_12      0
dtype: int64

In [43]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

100%|██████████| 16532648/16532648 [00:48<00:00, 343551.55it/s]

(14999, 42)


In [45]:
df_error_test = pd.DataFrame(test_x)
df_error_test['user_id'] = test_err['user_id']
df_error_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,user_id
0,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,0.0,1.0,0.0,983.0,1308.0,5.0,2.0,32.0,0.0,0.0,0.0,0.0,83.0,2.0,16.0,0.0,1.0,3.0,3.0,0.0,2.0,92.0,0.0,2.0,30000
1,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30000
2,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,0.0,0.0,60.0,40.0,2.0,0.0,40.0,0.0,0.0,0.0,0.0,230.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,14.0,113.0,1.0,2.0,30000
3,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,5.0,2.0,2.0,0.0,0.0,134.0,0.0,4.0,16.0,3.0,1.0,1.0,0.0,0.0,40.0,8.0,0.0,30000
4,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0.0,0.0,0.0,0.0,10.0,8.0,0.0,0.0,14.0,1.0,1.0,0.0,0.0,332.0,152.0,11.0,6.0,1.0,1.0,1.0,1.0,0.0,48.0,90.0,0.0,30000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,0.0,0.0,0.0,2.0,12.0,5.0,5.0,0.0,0.0,0.0,16.0,19.0,3.0,9.0,123.0,93.0,12.0,0.0,0.0,0.0,0.0,197.0,204.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,179.0,2.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,87.0,0.0,30015
14995,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,14.0,15.0,1.0,4.0,149.0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,183.0,0.0,14.0,1.0,0.0,1.0,1.0,0.0,0.0,12.0,17.0,0.0,30015
14996,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,12.0,13.0,1.0,3.0,129.0,90.0,2.0,0.0,0.0,0.0,0.0,138.0,165.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,163.0,0.0,12.0,1118.0,0.0,1.0,1.0,1.0,0.0,329.0,2.0,3.0,30015
14997,0.0,0.0,0.0,1.0,4.0,2.0,2.0,0.0,0.0,0.0,15.0,16.0,0.0,1.0,73.0,44.0,0.0,1.0,0.0,1.0,0.0,11127.0,13161.0,0.0,3.0,44.0,0.0,0.0,0.0,0.0,78.0,0.0,14.0,0.0,0.0,1.0,1.0,3.0,0.0,30.0,49.0,0.0,30015


In [46]:
merged_test_err_quality = pd.merge(df_error_test,mean_test_quality,on='user_id',how='left')
del merged_test_err_quality['user_id']
merged_test_err_quality.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,0.0,1.0,0.0,983.0,1308.0,5.0,2.0,32.0,0.0,0.0,0.0,0.0,83.0,2.0,16.0,0.0,1.0,3.0,3.0,0.0,2.0,92.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
1,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
2,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,0.0,0.0,60.0,40.0,2.0,0.0,40.0,0.0,0.0,0.0,0.0,230.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,14.0,113.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
3,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,5.0,2.0,2.0,0.0,0.0,134.0,0.0,4.0,16.0,3.0,1.0,1.0,0.0,0.0,40.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0.0,0.0,0.0,0.0,10.0,8.0,0.0,0.0,14.0,1.0,1.0,0.0,0.0,332.0,152.0,11.0,6.0,1.0,1.0,1.0,1.0,0.0,48.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0


In [ ]:
len(merged_test_err_quality)

14999

In [ ]:
merged_test_err_quality.isnull().sum()

0                0
1                0
2                0
3                0
4                0
5                0
6                0
7                0
8                0
9                0
10               0
11               0
12               0
13               0
14               0
15               0
16               0
17               0
18               0
19               0
20               0
21               0
22               0
23               0
24               0
25               0
26               0
27               0
28               0
29               0
30               0
31               0
32               0
33               0
34               0
35               0
36               0
37               0
38               0
39               0
40               0
41               0
quality_0     2292
quality_1     2292
quality_2     2292
quality_3     2292
quality_4     2292
quality_5     2292
quality_6     2292
quality_7     2292
quality_8     2292
quality_9     2292
quality_10  

In [47]:
multiple_imputation_df_test = pd.DataFrame(IterativeImputer(verbose=False).fit_transform(merged_test_err_quality))
multiple_imputation_df_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54
0,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,0.0,1.0,0.0,983.0,1308.0,5.0,2.0,32.0,0.0,0.0,0.0,0.0,83.0,2.0,16.0,0.0,1.0,3.0,3.0,0.0,2.0,92.0,0.0,2.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
1,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
2,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,0.0,0.0,60.0,40.0,2.0,0.0,40.0,0.0,0.0,0.0,0.0,230.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,14.0,113.0,1.0,2.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
3,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,5.0,2.0,2.0,0.0,0.0,134.0,0.0,4.0,16.0,3.0,1.0,1.0,0.0,0.0,40.0,8.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0.0,0.0,0.0,0.0,10.0,8.0,0.0,0.0,14.0,1.0,1.0,0.0,0.0,332.0,152.0,11.0,6.0,1.0,1.0,1.0,1.0,0.0,48.0,90.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,0.0,0.0,0.0,2.0,12.0,5.0,5.0,0.0,0.0,0.0,16.0,19.0,3.0,9.0,123.0,93.0,12.0,0.0,0.0,0.0,0.0,197.0,204.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,179.0,2.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,87.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.000000,1.833333,22.0,1.0,0.0,0.0,0.0,0.0
14995,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,14.0,15.0,1.0,4.0,149.0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,183.0,0.0,14.0,1.0,0.0,1.0,1.0,0.0,0.0,12.0,17.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.000000,1.833333,22.0,1.0,0.0,0.0,0.0,0.0
14996,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,12.0,13.0,1.0,3.0,129.0,90.0,2.0,0.0,0.0,0.0,0.0,138.0,165.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,163.0,0.0,12.0,1118.0,0.0,1.0,1.0,1.0,0.0,329.0,2.0,3.0,0.0,0.083333,0.0,0.0,0.0,0.000000,1.833333,22.0,1.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,0.0,1.0,4.0,2.0,2.0,0.0,0.0,0.0,15.0,16.0,0.0,1.0,73.0,44.0,0.0,1.0,0.0,1.0,0.0,11127.0,13161.0,0.0,3.0,44.0,0.0,0.0,0.0,0.0,78.0,0.0,14.0,0.0,0.0,1.0,1.0,3.0,0.0,30.0,49.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.000000,1.833333,22.0,1.0,0.0,0.0,0.0,0.0


In [48]:
pred_y_list = []
for model in models:
    pred_y = model.predict(multiple_imputation_df_test.values)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [49]:
pred_ensemble

array([[0.55957298],
       [0.27415445],
       [0.31373428],
       ...,
       [0.38225562],
       [0.60003271],
       [0.33856441]])

In [63]:
pred = np.where(pred_ensemble>0.35,1,0)
pred

array([[1],
       [0],
       [0],
       ...,
       [1],
       [1],
       [0]])

In [64]:
sample_submission['problem'] = pred
sample_submission.head()

,user_id,problem
0,30000,1
1,30001,0
2,30002,0
3,30003,1
4,30004,1


In [65]:
sample_submission.to_csv('/content/drive/MyDrive/데이콘/submission/lgb_baseline_merged_err_quality_imputation_submission_threshold0.35_ver2.csv')